# 영화 리뷰 데이터 Word2Vec로 분석

In [1]:
pip install Konlpy

     |████████████████████████████████| 19.4MB 43.1MB/s 
     |████████████████████████████████| 460kB 37.7MB/s 
     |████████████████████████████████| 92kB 5.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
from konlpy.tag import Komoran
import time #얼마나 학습 시간이 걸릴지

In [3]:
#네이버 영화 리뷰 데이터 읽어옴
def read_review_data(filename):
  with open(filename,'r') as f:
    #tab으로 구분
    data=[line.split('\t') for line in f.read().splitlines()]
    data=data[1:] #next역할
  
  return data


In [4]:
review_data=read_review_data('/content/ratings.txt')

In [5]:
len(review_data)

200000

In [6]:
#명사만 추출해 학습 입력 데이터 만들기
start = time.time()
print("형태소에서 명사만 추출 시작")
komoran = Komoran()
#한 줄식 받아 sentence[1]로 받아 추출 - nouns:명사만 추출
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print("형태소에서 명사만 추출 완료 : ",time.time() - start)

형태소에서 명사만 추출 시작
형태소에서 명사만 추출 완료 :  137.05464053153992


In [7]:
#word2vec 모델 학습 과정
from gensim.models import Word2Vec

#size는 단어임베딩의 차원의 크기 -> 보통 100~200
#text는 단어의 출연 빈도도 중요하다! -> min_count로 최소 2번은 나와야 포함한다.
#sg는 0과 1을 쓸 수 있는데 0은 CBOW, 1은 skip-gram
start = time.time()
print("모델학습 시작")
model=Word2Vec(sentences = docs, size=200, window=4, min_count=2, sg=0)
print("학습 소요 시간 : ",time.time()-start)
#backofwords를 사용하면 skip보다 학습속도가 빠르므로 16초 걸림

모델학습 시작
학습 소요 시간 :  16.886305332183838


In [8]:
start = time.time()
print("모델학습 시작")
model=Word2Vec(sentences = docs, size=200, window=4, min_count=2, sg=1)
print("학습 소요 시간 : ",time.time()-start)

모델학습 시작
학습 소요 시간 :  29.9112069606781


In [9]:
#모델 저장
model.save('/content/nvmc.model')

In [10]:
#학습된 말뭉치 개수
print("model.corpus_count: ", model.corpus_count)
#corpus 내 전체 단어 개수us_count)
#corpus 내 전체 단어 개수
print("model.corpus_total_words : ",model.corpus_total_words)
print("model.corpus_total_words : ",model.corpus_total_words)

model.corpus_count:  200000
model.corpus_total_words :  1076896
model.corpus_total_words :  1076896


In [11]:
#유사도 측정 방식
#단어 임베딩된 값과 벡터 공간상의 유사한 단어들을 확인

#모델 불러오기
model=Word2Vec.load('/content/nvmc.model')

In [12]:
#학습 잘 되었는지 확인
print("model.corpus_total_words: ",model.corpus_total_words)
#잘 되었다.

model.corpus_total_words:  1076896


In [13]:
print("사랑: ",model.wv['사랑'])
#얼추 200차원
#'사랑'이라는 단어의 벡터값을 출력

사랑:  [-0.4494772   0.456713   -0.47581106 -0.31735265 -0.08028495  0.2008743
 -0.00826496  0.14172262 -0.20021088  0.61448616  0.12031204  0.3562782
  0.6517951   0.41913086 -0.03490618  0.07787826  0.20880531  0.2013377
  0.00831773 -0.15666388  0.00175046 -0.02192963 -0.35989916  0.49388754
  0.1989361   0.07872412  0.04692243 -0.25156918 -0.02271424  0.3339236
  0.27113008  0.05537645  0.06206968  0.14147854 -0.00163235 -0.03187301
  0.06406411 -0.12247082 -0.37928206 -0.17455384  0.06576696 -0.07320766
 -0.05637597 -0.2395867  -0.13843271  0.09295412  0.03385063 -0.38770947
 -0.41767296 -0.3320322  -0.07067724 -0.01505921  0.08960607  0.16911177
  0.09139112 -0.06784994 -0.10870817 -0.12559067  0.2734048  -0.3734546
  0.0393755  -0.19306381  0.23761386  0.46850422 -0.25429764 -0.35232186
 -0.07603183  0.18129249  0.23963277 -0.20655312  0.00360907 -0.05253289
 -0.2899965   0.03448891  0.29845694  0.08349055 -0.12344032  0.08705919
  0.45427364  0.15579578 -0.30650204 -0.19587788 -0

In [14]:
#유사도 체크
#코사인 기반
#월요일과 일요일은 큰 유사도를 보여준다
print("일요일과 월요일의 유사도 : ",model.wv.similarity(w1="일요일", w2="월요일"))

일요일과 월요일의 유사도 :  0.9173818


In [15]:
print("사랑과 거짓말의 유사도 : ",model.wv.similarity(w1="사랑", w2="거짓말"))

사랑과 거짓말의 유사도 :  0.5350769


In [16]:
#wv-> Word2Vec
#topn -> 상위 몇 개 출력
print(model.wv.most_similar("일요일",topn=5))

[('출근', 0.9615507125854492), ('오전', 0.9524146914482117), ('금요일', 0.9439491033554077), ('밤새', 0.9352575540542603), ('회차', 0.9269595146179199)]


In [17]:
print(model.wv.most_similar("사랑",topn=5))

[('사랑은', 0.7429064512252808), ('치유', 0.7143785357475281), ('사랑으로', 0.7124640941619873), ('사람과', 0.7105830907821655), ('순수', 0.7080665826797485)]


In [18]:
print(model.wv.most_similar("거짓말",topn=5)) #코사인유사도 기반

[('안다', 0.8880045413970947), ('연락', 0.8822548389434814), ('다짐', 0.8820714950561523), ('나도', 0.8734508752822876), ('약속', 0.8732163906097412)]


In [ ]:
#유사도가 1에 가까울수록 좋다.

#이해하기 힘든 결과가 출력되는 경우는 주제에 맞는 말뭉치 데이터가 부족해서
#생길 수 있다. -> 품질 좋은 말뭉치 데이터를 학습하면 임베딩 성능 높아짐